In [1]:
import matplotlib.pyplot as plt

from scripts import tabular_baselines

import seaborn as sns
import numpy as np

from datasets import load_openml_list, valid_dids_classification, test_dids_classification, open_cc_dids
from scripts.tabular_baselines import *
from scripts.tabular_evaluation import evaluate
from scripts.tabular_metrics import calculate_score, make_ranks_and_wins_table, make_metric_matrix
from scripts import tabular_metrics
from scripts.transformer_prediction_interface import TabPFNClassifier
import matplotlib.pyplot as plt

from scripts import tabular_baselines

import seaborn as sns
import numpy as np

from datasets import load_openml_list, valid_dids_classification, test_dids_classification, open_cc_dids
from scripts.tabular_baselines import *
from scripts.tabular_evaluation import evaluate
from scripts.tabular_metrics import calculate_score, make_ranks_and_wins_table, make_metric_matrix
from scripts import tabular_metrics
from scripts.transformer_prediction_interface import TabPFNClassifier
from scripts.tabular_baselines import knn_metric, catboost_metric, transformer_metric, logistic_metric, xgb_metric, random_forest_metric


In [2]:
from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids, test_dids_classification

cc_test_datasets_multiclass, cc_test_datasets_multiclass_df = load_openml_list(open_cc_dids, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 10000, num_feats=100, return_capped=True)


Number of datasets: 30


In [3]:
cc_test_datasets_multiclass_df['isNumeric'] = (cc_test_datasets_multiclass_df.NumberOfSymbolicFeatures == 1) & (cc_test_datasets_multiclass_df.NumberOfInstancesWithMissingValues == 0)
name_indexed_dataset = cc_test_datasets_multiclass_df.reset_index().reset_index().set_index("name")

#numeric_test_datasets = [ds for ds in cc_test_datasets_multiclass if name_indexed_dataset.loc[ds[0]].isNumeric]
len(name_indexed_dataset)

30

In [4]:
len(cc_test_datasets_multiclass)

30

In [5]:
name_indexed_dataset.loc['eucalyptus']

level_0                                   12
index                                    188
did                                      188
version                                    1
uploader                                   1
status                                active
format                                  ARFF
MajorityClassSize                      214.0
MaxNominalAttDistinctValues             27.0
MinorityClassSize                      105.0
NumberOfClasses                          5.0
NumberOfFeatures                        20.0
NumberOfInstances                      736.0
NumberOfInstancesWithMissingValues      95.0
NumberOfMissingValues                  448.0
NumberOfNumericFeatures                 14.0
NumberOfSymbolicFeatures                 6.0
isNumeric                              False
Name: eucalyptus, dtype: object

In [6]:
# eucalyptus  188 

In [7]:
eucalyptus = [cc_test_datasets_multiclass[12]]

In [8]:
eucalyptus

[['eucalyptus',
  tensor([[ 3.0000,  2.0000,  3.0000,  ...,  4.0000,  4.5000,  4.0000],
          [13.0000,  3.0000,  0.0000,  ...,     nan,     nan,     nan],
          [ 6.0000,  2.0000,  4.0000,  ...,  3.3000,  3.3000,  3.3000],
          ...,
          [ 2.0000,  3.0000,  2.0000,  ...,  4.0000,  4.0000,  3.0000],
          [ 4.0000,  2.0000,  4.0000,  ...,  3.5000,  4.5000,  4.0000],
          [ 7.0000,  1.0000,  2.0000,  ...,  2.5000,  2.5000,  2.0000]]),
  tensor([0, 0, 1, 1, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 4, 0, 0, 3, 4, 2, 2, 4, 2, 2,
          3, 1, 4, 0, 0, 2, 1, 0, 0, 0, 3, 1, 3, 3, 3, 0, 4, 2, 3, 3, 1, 2, 0, 1,
          3, 2, 4, 3, 4, 4, 1, 0, 4, 1, 2, 3, 2, 1, 2, 1, 4, 2, 0, 3, 0, 4, 0, 3,
          1, 3, 4, 3, 0, 4, 4, 0, 1, 3, 2, 3, 0, 3, 0, 1, 1, 0, 1, 0, 3, 0, 1, 0,
          3, 2, 0, 2, 1, 3, 4, 3, 1, 0, 2, 1, 3, 2, 3, 3, 2, 3, 1, 3, 0, 3, 2, 3,
          1, 0, 0, 0, 3, 3, 3, 2, 0, 4, 0, 4, 3, 3, 4, 2, 0, 4, 0, 1, 1, 3, 3, 2,
          0, 2, 3, 3, 3, 3, 3, 4, 4, 1, 3, 

In [9]:
eucalyptus_filled_nan = [[['eucalyptus-filled-nan'], torch.nan_to_num(eucalyptus[0][1], 0)] + eucalyptus[0][2:]]

In [10]:
from tabpfn.scripts.tabular_evaluation import evaluate
from functools import partial

eval_positions = [1000]
max_times = [1]
metric_used = tabular_metrics.auc_metric
bptt = 2000
base_path = os.path.join('.')

tabpfn = TabPFNClassifier()
results_transformers_ohe_false = [evaluate(eucalyptus, model=partial(transformer_metric, classifier=tabpfn), eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_number=i, save=False, path_interfix="bla", method="transformer-grrrr-9",
                     bptt=bptt, base_path=base_path, overwrite=False) for i in range(1, 2)]


/anaconda/envs/tabpfn_testing_environment/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using a Transformer with 25.82 M parameters
Using cpu device


In [11]:
tabpfn = TabPFNClassifier()
results_transformers_ohe_false = [evaluate(eucalyptus_filled_nan, model=partial(transformer_metric, classifier=tabpfn), eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_number=i, save=False, path_interfix="bla", method="transformer-grrrr-9",
                     bptt=bptt, base_path=base_path, overwrite=False) for i in range(1, 2)]


> /home/azureuser/TabPFN/tabpfn/scripts/transformer_prediction_interface.py(406)predict()
    404         # Initialize results array size S, B, Classes
    405         import pdb; pdb.set_trace()
--> 406         inference_mode_call = torch.inference_mode() if inference_mode else NOP()
    407         with inference_mode_call:
    408             start = time.time()



ipdb>  p eval_xs.isnan().any()


tensor(False)


ipdb>  p eval_xs.shape


torch.Size([736, 3, 100])


...

ipdb>  p eval_xs.mean()


tensor(0.0004)


ipdb>  p eval_xs.std()


tensor(2.2772)


ipdb>  exit


In [ ]:
from tabpfn.scripts.model_builder import get_encoder
tabpfn.model.encoder.replace_nan_by_zero

In [32]:
pd.DataFrame(results_transformers_ohe_false).mean_metric.mean()

0.7841577053070068

In [12]:
pd.DataFrame(results_transformers).mean_metric

0    tensor(0.7157)
1    tensor(0.8332)
2    tensor(0.8175)
3    tensor(0.7167)
4    tensor(0.8377)
Name: mean_metric, dtype: object

In [47]:
from functools import partial
results_32 = eval_on_datasets('multiclass', partial(transformer_metric, N_ensemble_configurations=32), 'transformer_bla_32', eucalyptus, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     bptt=bptt, base_path=base_path, overwrite=False, n_jobs=-1)

evaluating transformer_bla_32 on cpu


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   23.2s remaining:   15.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   23.7s finished


In [48]:
pd.DataFrame(results_32).mean_metric

0    tensor(0.7126)
1    tensor(0.8384)
2    tensor(0.8241)
3    tensor(0.7194)
4    tensor(0.8375)
Name: mean_metric, dtype: object

In [51]:
results_3 = eval_on_datasets('multiclass', partial(transformer_metric, N_ensemble_configurations=3), 'transformer_bla_3', eucalyptus, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     bptt=bptt, base_path=base_path, overwrite=False, n_jobs=-1)

evaluating transformer_bla_3 on cpu


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    5.3s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.4s finished


In [53]:
pd.DataFrame(results_3).mean_metric

0    tensor(0.7157)
1    tensor(0.8332)
2    tensor(0.8175)
3    tensor(0.7167)
4    tensor(0.8377)
Name: mean_metric, dtype: object

In [54]:
eucalyptus

[['eucalyptus',
  tensor([[ 3.0000,  2.0000,  3.0000,  ...,  4.0000,  4.5000,  4.0000],
          [13.0000,  3.0000,  0.0000,  ...,     nan,     nan,     nan],
          [ 6.0000,  2.0000,  4.0000,  ...,  3.3000,  3.3000,  3.3000],
          ...,
          [ 2.0000,  3.0000,  2.0000,  ...,  4.0000,  4.0000,  3.0000],
          [ 4.0000,  2.0000,  4.0000,  ...,  3.5000,  4.5000,  4.0000],
          [ 7.0000,  1.0000,  2.0000,  ...,  2.5000,  2.5000,  2.0000]]),
  tensor([0, 0, 1, 1, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 4, 0, 0, 3, 4, 2, 2, 4, 2, 2,
          3, 1, 4, 0, 0, 2, 1, 0, 0, 0, 3, 1, 3, 3, 3, 0, 4, 2, 3, 3, 1, 2, 0, 1,
          3, 2, 4, 3, 4, 4, 1, 0, 4, 1, 2, 3, 2, 1, 2, 1, 4, 2, 0, 3, 0, 4, 0, 3,
          1, 3, 4, 3, 0, 4, 4, 0, 1, 3, 2, 3, 0, 3, 0, 1, 1, 0, 1, 0, 3, 0, 1, 0,
          3, 2, 0, 2, 1, 3, 4, 3, 1, 0, 2, 1, 3, 2, 3, 3, 2, 3, 1, 3, 0, 3, 2, 3,
          1, 0, 0, 0, 3, 3, 3, 2, 0, 4, 0, 4, 3, 3, 4, 2, 0, 4, 0, 1, 1, 3, 3, 2,
          0, 2, 3, 3, 3, 3, 3, 4, 4, 1, 3, 

Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device
Using a Transformer with 25.82 M parameters
Using cpu device


In [ ]:
results_32_try_again = eval_on_datasets('multiclass', partial(transformer_metric, N_ensemble_configurations=3), 'transformer_bla_32', eucalyptus, eval_positions=eval_positions, max_times=max_times,
                     metric_used=metric_used, split_numbers=[1, 2, 3, 4, 5],
                     bptt=bptt, base_path=base_path, overwrite=False, n_jobs=-1)